## Red Neuronal MultiCapa, OOP model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

### Importar Images

In [2]:
from get_images import get_images

In [3]:
# MNIST path
mnist_path = './mnist_raw/'
x_train_num, y_train_num, x_test_num, y_test_num = get_images(mnist_path)

x_train = x_train_num[:50000].reshape(50000, -1).astype(float)
y_train = y_train_num[:50000].reshape(50000, 1)

x_val = x_train_num[50000:].reshape(10000, -1).astype(float)
y_val = y_train_num[50000:].reshape(10000, 1)

x_test = x_test_num.copy().reshape(10000, -1).astype(float)
y_test = y_test_num.copy().reshape(10000, 1)

### Normalizar imágenes

In [4]:
x_train.mean(), x_train.std(), x_train.min()

(33.39512885204082, 78.6661972212754, 0.0)

In [5]:
def normalise(x_mean, x_std, x_data):
    return (x_data - x_mean) / x_std

In [6]:
x_mean = x_train.mean()
x_std = x_train.std()

x_train = normalise(x_mean, x_std, x_train)
x_val = normalise(x_mean, x_std, x_val)
x_test = normalise(x_mean, x_std, x_test)

In [7]:
x_train.mean(), x_train.std()

(-9.646705203355238e-18, 0.9999999999999997)

In [8]:
x_test.shape

(10000, 784)

### Graficar muestras

In [9]:
def plot_number(image):
    plt.figure(figsize=(5,5))
    plt.imshow(image.squeeze(), cmap=plt.get_cmap('gray'))
    plt.axis('off')
    plt.show()

In [10]:
#rnd_idx = np.random.randint(len(y_test))
#print(f'La imagen muestreada representa un: {y_test[rnd_idx, 0]}')
#plot_number(x_test_num[rnd_idx])

### Ecuaciones para nuestro modelo


$$z^1 = W^1 X + b^1$$

$$a^1 = ReLU(z^1) $$

$$z^2 = W^2 a^1 + b^2$$

$$\hat{y} = \frac{e^{z^{2_k}}}{\sum_j{e^{z_j}}}$$


$$ \mathcal{L}(\hat{y}^{i}, y^{i}) =  - y^{i}  \ln(\hat{y}^{i}) = -\ln(\hat{y}^i)$$


$$ \mathcal{J}(w, b) =  \frac{1}{num\_samples} \sum_{i=1}^{num\_samples}-\ln(\hat{y}^{i})$$

### Crear Minibatches

In [11]:


def create_minibatches(x, y, mb_size, shuffle = True):
    '''
    x  #muestras, 784
    y #muestras, 1
    '''
    assert x.shape[0] == y.shape[0], 'Error en cantidad de muestras'
    total_data = x.shape[0]
    if shuffle: 
        idxs = np.arange(total_data)
        np.random.shuffle(idxs)
        x = x[idxs]
        y = y[idxs]  
    return ((x[i:i+mb_size], y[i:i+mb_size]) for i in range(0, total_data, mb_size))



### Convertir Numpy array a PyTorch

In [12]:
x_train_tensor = torch.tensor(x_train.copy())
y_train_tensor = torch.tensor(y_train.copy())

x_val_tensor = torch.tensor(x_val.copy())
y_val_tensor = torch.tensor(y_val.copy())

x_test_tensor = torch.tensor(x_test.copy())
y_test_tensor = torch.tensor(y_test.copy())

### Usar GPU de estar disponible

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Estammos usando: {device}')



Estammos usando: cpu


### Compute accuracy

In [16]:
def accuracy(model, x, y, mb_size):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for (xi, yi) in create_minibatches(x, y, mb_size):
            xi = xi.to(device=device, dtype = torch.float32)
            yi = yi.to(device=device, dtype = torch.long)
            scores = model(xi) #[mb_size, 10]
            _, pred = scores.max(dim=1)
            num_correct += (pred == yi.squeeze()).sum() # pred shape(mb_size), yi shape(mb_size, 1)
            num_total += pred.size(0)

            return float(num_correct)/num_total
            

### Loop de entrenamiento

In [19]:
def train(model, optimiser, mb_size, epochs=100):
    model = model.to(device=device)
    for epoch in range(epochs):
        for (xi, yi) in create_minibatches(x_train_tensor, y_train_tensor, mb_size):
            model.train()
            xi = xi.to(device=device, dtype=torch.float32)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)
            #function cost
            cost = F.cross_entropy(input = scores, target = yi.squeeze())
            optimiser.zero_grad()
            cost.backward()
            optimiser.step()
            
        print(f'Epoch: {epoch}, costo: {cost.item()}, accuracy: {accuracy(model, x_val_tensor, y_val_tensor, mb_size)}')
            

### Modelo usando Sequencial#Instanciar modelo
#model = nn.Sequencial(Linear)

In [26]:
#Instanciar modelo
hidden1 = 1000
hidden = 1000
lr = 5e-2
epochs = 10
mb_size = 4096

#model1 = nn.Sequential(nn.Linear(in_features=784, out_features=hidden1), nn.ReLU(),
#                       nn.Linear(in_features=hidden1, out_features=hidden), nn.ReLU(),
#                       nn.Linear(in_features=hidden, out_features=10))
optimiser = torch.optim.SGD(model1.parameters(), lr=lr)

train(model1, optimiser, mb_size, epochs)

Epoch: 0, costo: 0.30391550064086914, accuracy: 0.912109375
Epoch: 1, costo: 0.251731812953949, accuracy: 0.913330078125
Epoch: 2, costo: 0.31207016110420227, accuracy: 0.923095703125
Epoch: 3, costo: 0.29431289434432983, accuracy: 0.920166015625
Epoch: 4, costo: 0.29853981733322144, accuracy: 0.918212890625
Epoch: 5, costo: 0.2434449940919876, accuracy: 0.9267578125
Epoch: 6, costo: 0.3196466565132141, accuracy: 0.9208984375
Epoch: 7, costo: 0.28594210743904114, accuracy: 0.929443359375
Epoch: 8, costo: 0.24647089838981628, accuracy: 0.93212890625
Epoch: 9, costo: 0.2749399244785309, accuracy: 0.93017578125


In [27]:
accuracy(model1, x_test_tensor, y_test_tensor, mb_size)

0.930419921875